<a href="https://colab.research.google.com/github/Ditsuhi/ExploratoryAnalysis_FeatureSelection/blob/main/Copy_of_BiConvLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import all required libraries

import pandas as pd
import numpy as np
from time import time
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import ConvLSTM2D, BatchNormalization, Dropout, Bidirectional, Conv2D 

In [2]:
# the dataset can be found at the following link: https://doi.org/10.5281/zenodo.6497108. 
# the path provided below can be changed depending your data location.

datafr_2019 = pd.read_csv('/content/Madrid_wind_2019.csv')
datafr_2020 = pd.read_csv('/content/Madrid_wind_2020.csv', index_col='Unnamed: 0')

In [3]:
datafr_2020.columns

Index(['NO2', 'windSpeed', 'Temp', 'Humidity', 'Pressure', 'SolarRad',
       'intensidad', 'ocupacion', 'carga', 'vmed', 'v_comp', 'u_comp',
       'windDir_Categ_east', 'windDir_Categ_north', 'windDir_Categ_northeast',
       'windDir_Categ_northwest', 'windDir_Categ_south',
       'windDir_Categ_southeast', 'windDir_Categ_southwest',
       'windDir_Categ_west'],
      dtype='object')

In [4]:
datafr_2019.columns

Index(['NO2', 'windSpeed', 'Temp', 'Humidity', 'Pressure', 'SolarRad',
       'intensidad', 'ocupacion', 'carga', 'vmed', 'v_comp', 'u_comp',
       'windDir_Categ_east', 'windDir_Categ_north', 'windDir_Categ_northeast',
       'windDir_Categ_northwest', 'windDir_Categ_south',
       'windDir_Categ_southeast', 'windDir_Categ_southwest',
       'windDir_Categ_west'],
      dtype='object')

In [15]:
# this part depends on the selected features, extracted after mutual information and mRMR implementation.
# Below are the features for each scenario after implementing feature selection techniques



# # Scenario_First_All_Features
# datafr_new_2019=datafr_2019[['NO2', 'windSpeed', 'Temp', 'Humidity', 'Pressure', 'SolarRad',
#        'intensidad', 'ocupacion', 'carga', 'vmed', 'windDir_Categ_east', 'windDir_Categ_north', 'windDir_Categ_northeast',
#        'windDir_Categ_northwest', 'windDir_Categ_south',
#        'windDir_Categ_southeast', 'windDir_Categ_southwest',
#        'windDir_Categ_west']
#       ]
     

# datafr_new_2020=datafr_2020[['NO2', 'windSpeed', 'Temp', 'Humidity', 'Pressure', 'SolarRad',
#        'intensidad', 'ocupacion', 'carga', 'vmed', 'windDir_Categ_east', 'windDir_Categ_north', 'windDir_Categ_northeast',
#        'windDir_Categ_northwest', 'windDir_Categ_south',
#        'windDir_Categ_southeast', 'windDir_Categ_southwest',
#        'windDir_Categ_west']]


# # Scenario_Second_All_Features
# datafr_new_2019=datafr_2019[['NO2', 'windSpeed', 'Temp', 'Humidity', 'Pressure', 'SolarRad',
#        'intensidad', 'ocupacion', 'carga', 'vmed', 'v_comp', 'u_comp']]
     

# datafr_new_2020=datafr_2020[['NO2', 'windSpeed', 'Temp', 'Humidity', 'Pressure', 'SolarRad',
#        'intensidad', 'ocupacion', 'carga', 'vmed', 'v_comp', 'u_comp']]



# # Scenario_First_MI 
# datafr_new_2019=datafr_2019[['NO2', 'windSpeed',  'Pressure', 'intensidad', 'ocupacion', 'carga', 'vmed']]
     
# datafr_new_2020=datafr_2020[['NO2', 'windSpeed',  'Pressure', 'intensidad', 'ocupacion', 'carga', 'vmed']]



# # Scenario_Second_MI 
# datafr_new_2019=datafr_2019[['NO2', 'windSpeed',  'Pressure', 'intensidad', 'ocupacion', 'carga', 'vmed', 'v_comp', 'u_comp']]
     
# datafr_new_2020=datafr_2020[['NO2', 'windSpeed',  'Pressure', 'intensidad', 'ocupacion', 'carga', 'vmed', 'v_comp', 'u_comp']]


# # Scenario_First_mRMR (K=8). Here, the characteristics are written in descending order of importance of predicting nitrogen dioxide. 
# # For example, if K=3, we should extract the first 4 features since NO2 is the first feature and the other 3 are the most relevant features.
# #'carga', 'windDir_Categ_northwest', 'Pressure', 'windSpeed', 'vmed', 'ocupacion', 'windDir_Categ_north', 'intensidad'
# datafr_new_2019=datafr_2019[['NO2', 'carga', 'windDir_Categ_northwest', 'Pressure']]
     
# datafr_new_2020=datafr_2020[['NO2', 'carga', 'windDir_Categ_northwest', 'Pressure']]


# Scenario_Second_mRMR (K=8). Here, the characteristics are written in descending order of importance of predicting nitrogen dioxide. 
# For example, if K=3, we should extract the first 4 features since NO2 is the first feature and the other 3 are the most relevant features.
# 'NO2', 'carga', 'Pressure', 'windSpeed', 'vmed', 'ocupacion', 'SolarRad', 'intensidad', 'Temp'
datafr_new_2019=datafr_2019[['NO2', 'carga', 'Pressure', 'windSpeed', 'vmed', 'ocupacion', 'SolarRad', 'intensidad', 'Temp']]
     
datafr_new_2020=datafr_2020[['NO2', 'carga', 'Pressure', 'windSpeed', 'vmed', 'ocupacion', 'SolarRad', 'intensidad', 'Temp']]




In [16]:
len(datafr_new_2020.columns)

9

In [17]:
# convert dataframes to numpy array and reshape them. 

# number of selected features
number_selected_columns =9

data_np_2019 = np.asarray(datafr_new_2019)
data_np_2020 = np.asarray(datafr_new_2020)
data_np_2019_resh = data_np_2019.reshape(-1, 340, number_selected_columns)
data_np_2020_resh = data_np_2020.reshape(-1, 340, number_selected_columns)

# define training, validation and testing sets
data_2019_train = data_np_2019_resh[:, :, :]
data_2020_val = data_np_2020_resh[0:2184, :, :]
data_2020_test = data_np_2020_resh[2184::, :, :]

In [18]:
# split dataset to X and y (dependent and independent)

def split_sequence(sequence, time_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
   
		# find the end of this pattern
		end_ix = i + 6
    
		# check if we are beyond the sequence
		if end_ix+time_steps > len(sequence)-1:
			break
		# gather input and output parts of the pattern    
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix: end_ix+time_steps]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
 


# choose a number of time steps and define dependent and independent sets.
time_steps = 6
X_train_notNorm, y_train = split_sequence(data_2019_train, time_steps)
X_val_notNorm, y_val = split_sequence(data_2020_val, time_steps)
X_test_notNorm, y_test = split_sequence(data_2020_test, time_steps)


In [19]:
# to normalise train data using MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1), copy = False)
X_train_Normalised = X_train_notNorm.reshape(-1, number_selected_columns)
X_val_Normalised = X_val_notNorm.reshape(-1, number_selected_columns)
X_test_Normalised = X_test_notNorm.reshape(-1, number_selected_columns)

X_train_scaled = scaler.fit_transform(X_train_Normalised)
X_val_scaled = scaler.transform(X_val_Normalised)
X_test_scaled = scaler.transform(X_test_Normalised)

X_train = X_train_scaled.reshape(X_train_notNorm.shape[0], X_train_notNorm.shape[1], X_train_notNorm.shape[2], X_train_notNorm.shape[3])
X_val = X_val_scaled.reshape(X_val_notNorm.shape[0], X_val_notNorm.shape[1], X_val_notNorm.shape[2], X_val_notNorm.shape[3])
X_test = X_test_scaled.reshape(X_test_notNorm.shape[0], X_test_notNorm.shape[1], X_test_notNorm.shape[2], X_test_notNorm.shape[3])

In [20]:
# to reshape data corresponding to the input data of BiConvLSTM method.

X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 20, 17*number_selected_columns, 1))
y_train_reshaped = y_train.reshape((y_train.shape[0], y_train.shape[1], 20, 17*number_selected_columns, 1))
X_val_reshaped = X_val.reshape((X_val.shape[0], X_val.shape[1], 20, 17*number_selected_columns, 1))
y_val_reshaped = y_val.reshape(y_val.shape[0], y_val.shape[1], 20, 17*number_selected_columns, 1)
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 20, 17*number_selected_columns, 1))
y_test_reshaped = y_test.reshape(y_test.shape[0], y_test.shape[1], 20, 17*number_selected_columns, 1)

In [21]:
y_test_reshaped.shape

(2171, 6, 20, 153, 1)

In [12]:
# run trained models using .json and .h5 files for each subsets (extracted features for each scenario implementing mutual information and mRMR)

from keras.models import model_from_json
json_file = open('/content/Scenario_Second_mRMR_(K=8).json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/Scenario_Second_mRMR_(K=8).h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='mse')


Loaded model from disk


In [13]:
# evaluate the model
 
# perform prediction
yhat = loaded_model.predict(X_test_reshaped, verbose=1)
# reshape the predicted results and the test data for further evaluation
yhat_reshaped = yhat.reshape(-1,20*17*number_selected_columns)
y_test_reshaped=  y_test_reshaped.reshape(-1,20*17*number_selected_columns)
# calculate RMSE and MAE
rmse = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
mae = mean_absolute_error(yhat_reshaped, y_test_reshaped)
print('Test Score: %.2f RMSE' % (rmse))
print('Test Score: %.2f MAE' % (mae))

68/68 [==============================] - 15s 165ms/step
Test Score: 31.80 RMSE
Test Score: 21.77 MAE


Here is the implemented model with complete architecture. The model can run separately without the trained files. It should be noted that in this case, the results may vary slightly due to the existing randomness in stochastic machine learning algorithms, and this may also be caused by the development environment, considering different software versions and CPU types.

In [22]:
# define the architecture of the proposed model.

opt = keras.optimizers.Adam(learning_rate=0.01)
def create_model(number_selected_columns=number_selected_columns, optimizer=opt, kernel_size=(3, 3), filters=16, merge_mode="concat", dropout_rate=0.2):
    
    model = Sequential()    
    model.add(Bidirectional(ConvLSTM2D(input_shape=(None, 20, 17*number_selected_columns, 1),  filters=filters,  kernel_size=kernel_size, padding="same", return_sequences=True), merge_mode=merge_mode))
    
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))   
    model.add(Bidirectional(ConvLSTM2D(filters=filters, kernel_size=kernel_size, padding="same", return_sequences=True), merge_mode=merge_mode))    
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate)) 
    model.add(Bidirectional(ConvLSTM2D(filters=filters,  kernel_size=kernel_size, padding="same", return_sequences=True), merge_mode=merge_mode))     
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))            
    model.add(Conv2D(filters=1, kernel_size=(1, 1), 
                activation='elu',
                padding='same', data_format='channels_last'))
    model.compile(optimizer=optimizer, loss='mse')
    model.build(input_shape=(None,6,  20, 17*number_selected_columns, 1))    
    print(model.summary())
    return model

In [23]:
# run the model

model = create_model()
start = time()
early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=2)
final_model = model.fit(X_train_reshaped, y_train_reshaped, epochs=50, verbose=2, validation_data=(X_val_reshaped, y_val_reshaped),  callbacks=[early_stopping])
print(f'Time taken to run: {time() - start} seconds')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 6, 20, 153, 32)   19712     
 nal)                                                            
                                                                 
 batch_normalization_3 (Batc  (None, 6, 20, 153, 32)   128       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 6, 20, 153, 32)    0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 6, 20, 153, 32)   55424     
 nal)                                                            
                                                                 
 batch_normalization_4 (Batc  (None, 6, 20, 153, 32)   128       
 hNormalization)                                      

In [24]:
# evaluate the model
 
# perform prediction
yhat = model.predict(X_test_reshaped, verbose=1)
# reshape the predicted results and the test data for further evaluation
yhat_reshaped = yhat.reshape(-1,20*17*number_selected_columns)
y_test_reshaped=  y_test_reshaped.reshape(-1,20*17*number_selected_columns)
# calculate RMSE and MAE
rmse = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
mae = mean_absolute_error(yhat_reshaped, y_test_reshaped)
print('Test Score: %.2f RMSE' % (rmse))
print('Test Score: %.2f MAE' % (mae))

68/68 [==============================] - 14s 175ms/step
Test Score: 30.66 RMSE
Test Score: 19.39 MAE


In [ ]:
# here is an example of saving and loading trained files

from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("Scenario_Second_mRMR_(K=2).json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Scenario_Second_mRMR_(K=2).h5")
print("Saved model to disk")
 

# load json and create model
json_file = open('Scenario_Second_mRMR_(K=2).json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Scenario_Second_mRMR_(K=2).h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(optimizer=opt, loss='mse')


Saved model to disk
Loaded model from disk
